In [2]:
import numpy as np
import os
import pandas as pd
import arcpy

In [3]:
working_directory = os.getcwd()
print(working_directory)

zipcode_shp = os.path.join(working_directory + r"\ZipCode\zipcode.shp")
taz_shp = working_directory + r"\TAZ\TAZ.shp"
                        
taz_table = working_directory + r"\TAZ\TAZ.dbf"
tazcentroids_shp = working_directory + r"\TAZ\TAZCentroids.shp"

tazcentroids_zipcode_sj = working_directory + r"\Results\Shapefile\TAZCentroids_ZipCode.shp"
taz_zipcode_shp = working_directory + r"\Results\Shapefile\TAZ_ZipCode.shp"

taz_zipcode_csv = working_directory + r"\Results\CSV\TAZ_ZipCode.csv"

E:\GitHub\TAZ-to-ZIPCode


In [33]:
sde_filename = "agrc_opensgid"

#make only once
arcpy.CreateDatabaseConnection_management (working_directory, sde_filename, "POSTGRESQL", "opensgid.agrc.utah.gov", "DATABASE_AUTH", "agrc", "agrc", "SAVE_USERNAME", "opensgid")

In [34]:
zipcodearea_layername = "opensgid.boundaries.zip_code_areas"

zipcode = sde_filename + ".sde\\" + zipcodearea_layername

print(arcpy.Describe(zipcode).shapeType)



Polygon


<Result 'opensgid.boundaries.zip_code_areas'>

Copy locally to make further processing faster

In [42]:
arcpy.CopyFeatures_management(zipcode, zipcode_fc)

<Result 'E:\\GitHub\\TAZ-to-ZIPCode\\ZipCode\\ZipCode.shp'>

In [5]:
arcpy.MakeFeatureLayer_management(zipcode_fc + ".shp","ZC")

<Result 'ZC'>

In [6]:
arcpy.MakeFeatureLayer_management(working_directory + "\\TAZ\\TAZ.shp", "TAZ")

<Result 'TAZ'>

In [70]:
# Set the local variables

x_coords = "X"
y_coords = "Y"

print (taz_fc)
spatialRef = arcpy.Describe(taz_fc).spatialReference

# Make the XY event layer...
arcpy.management.XYTableToPoint(taz_table, tazcentroids_shp,
                                x_coords, y_coords, None, spatialRef)


E:\GitHub\TAZ-to-ZIPCode\TAZ\TAZ.shp


<Result 'E:\\GitHub\\TAZ-to-ZIPCode\\TAZ\\TAZCentroids.shp'>

In [71]:
arcpy.SpatialJoin_analysis(tazcentroids_shp, zipcode_shp, tazcentroids_zipcode_sj, "JOIN_ONE_TO_ONE", "", "", "CLOSEST", "", "")

<Result 'E:\\GitHub\\TAZ-to-ZIPCode\\Results\\Shapefile\\TAZCentroids_ZipCode.shp'>

In [74]:
# Create the required FieldMap and FieldMappings objects
fm_taz = arcpy.FieldMap()
fm_zip = arcpy.FieldMap()
fms = arcpy.FieldMappings()

# Get the field names of vegetation type and diameter for both original
# files
tazfieldname = "TAZID"
zipfieldname = "zip5"

# Add fields to their corresponding FieldMap objects
fm_taz.addInputField(tazcentroids_shp, tazfieldname)
fm_zip.addInputField(zipcode_shp, zipfieldname)

# Set the output field properties for both FieldMap objects
taz_name = fm_taz.outputField
taz_name.name = 'TAZID'
fm_taz.outputField = taz_name

zip_name = fm_zip.outputField
zip_name.name = 'ZIP5'
fm_zip.outputField = zip_name

# Add the FieldMap objects to the FieldMappings object
fms.addFieldMap(fm_taz)
fms.addFieldMap(fm_zip)


arcpy.TableToTable_conversion(tazcentroids_zipcode_sj, os.path.dirname(taz_zipcode_csv), os.path.basename(taz_zipcode_csv), field_mapping=fms)

<Result 'E:\\GitHub\\TAZ-to-ZIPCode\\Results\\CSV\\TAZ_ZipCode.csv'>

In [5]:
from arcpy import FeatureClassToFeatureClass_conversion
from arcpy import AddJoin_management
from arcpy import MakeTableView_management
from arcpy import MakeFeatureLayer_management

taz_layer = MakeFeatureLayer_management(taz_shp,"taz")
zip_layer = MakeTableView_management(taz_zipcode_csv, "zip")

taz_zip = AddJoin_management(taz_layer, 'TAZID', zip_layer, 'TAZID')
FeatureClassToFeatureClass_conversion(taz_zip, os.path.dirname(taz_zipcode_shp), os.path.basename(taz_zipcode_shp))

<Result 'E:\\GitHub\\TAZ-to-ZIPCode\\Results\\Shapefile\\TAZ_ZipCode.shp'>

In [12]:
from arcgis.gis import GIS
gis = GIS()

In [18]:
working_directory + r"\TAZ\TAZ.shp"

'E:\\GitHub\\TAZ-to-ZIPCode\\TAZ\\TAZ.shp'

In [19]:
#Create a new map object
map1 = gis.map()
#Focus the map to the part of the world containing the example item
#map1.extent = example_item.extent
#Display the map
map1

map1.add_layer(working_directory + r"\TAZ\TAZ.shp")

RuntimeError: Cannot infer layer: will not be added to map